## Making a Stock Screener with Python!

* Link: https://towardsdatascience.com/making-a-stock-screener-with-python-4f591b198261
* Summary: Uses 7 different conidtions on stocks current values to return a dataframe which shows all the stocks that passed or meet the requirements.

### Screen Criteria:
1. The current price of the security must be greater than the 150 and 200-day simple moving averages.
2. The 150-day simple moving average must be greater than the 200-day simple moving average.
3. The 200-day simple moving average must be trending up for at least 1 month.
4. The 50-day simple moving average must be greater than the 150 simple moving average and the 200 simple moving average.
5. The current price must be greater than the 50-day simple moving average.
6. The current price must be at least 30% above the 52 week low.
7. The current price must be within 25% of the 52 week high.
8. The IBD RS-Rating must be greater than 70 (the higher, the better). The RS rating is a metric of a stock’s price performance over the last year compared to all other stocks and the overall market. Check out this article to learn more.

In [1]:
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
from pandas import ExcelWriter
import yfinance as yf
import pandas as pd
import datetime
import time
yf.pdr_override()

First, we define all our variables.
* tickers : All of the tickers in the S&P 500
* index_name : S&P 500 Yahoo Finance symbol
* start_date : The beginning date for historical data (exactly one year ago)
* end_date : The end date for historical data (today)
* exportList : The values we will be collecting for each stock
* returns_multiples : A list to see how each stock performed relative to the market (will be used for calculating the RS rating)

In [2]:
tickers = si.tickers_sp500()
tickers = [item.replace(".", "-") for item in tickers] # Yahoo Finance uses dashes instead of dots
index_name = '^GSPC' # S&P 500
start_date = datetime.datetime.now() - datetime.timedelta(days=365)
end_date = datetime.date.today()
exportList = pd.DataFrame(columns=['Stock', "RS_Rating", "50 Day MA", "150 Day Ma", "200 Day MA", "52 Week Low", "52 week High"])
returns_multiples = []

Now, we can calculate the cumulative return of the S&P 500 index over the past year and compare that value to the cumulative return for each stock in the S&P 500 during the same period.

In [ ]:
index_df = pdr.get_data_yahoo(index_name, start_date, end_date)
index_df['Percent Change'] = index_df['Adj Close'].pct_change()
index_return = (index_df['Percent Change'] + 1).cumprod()[-1]

After we get the S&P 500 index returns, we compare it to all the stocks to find the top 30% compared to the S&P 500 index.

In [4]:
for ticker in tickers:
    # Download historical data as CSV for each stock (makes the process faster)
    df = pdr.get_data_yahoo(ticker, start_date, end_date)
    df.to_csv(f'{ticker}.csv')

    # Calculating returns relative to the market (returns multiple)
    df['Percent Change'] = df['Adj Close'].pct_change()
    stock_return = (df['Percent Change'] + 1).cumprod()[-1]
    
    returns_multiple = round((stock_return / index_return), 2)
    returns_multiples.extend([returns_multiple])
    
    
    time.sleep(1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Once, we find the top 30%, we can store those stocks in a dataframe. 

In [5]:
rs_df = pd.DataFrame(list(zip(tickers, returns_multiples)), columns=['Ticker', 'Returns_multiple'])
rs_df['RS_Rating'] = rs_df.Returns_multiple.rank(pct=True) * 100
rs_df = rs_df[rs_df.RS_Rating >= rs_df.RS_Rating.quantile(.70)]

Finally, taking the top 30% stocks, we check each of the stocks included against the 7 conditions listed to screen them further and know which one passes all 7. 

In [6]:
rs_stocks = rs_df['Ticker']
for stock in rs_stocks:    
    try:
        df = pd.read_csv(f'{stock}.csv', index_col=0)
        sma = [50, 150, 200]
        for x in sma:
            df["SMA_"+str(x)] = round(df['Adj Close'].rolling(window=x).mean(), 2)
        
        # Storing required values 
        currentClose = df["Adj Close"][-1]
        moving_average_50 = df["SMA_50"][-1]
        moving_average_150 = df["SMA_150"][-1]
        moving_average_200 = df["SMA_200"][-1]
        low_of_52week = round(min(df["Low"][-260:]), 2)
        high_of_52week = round(max(df["High"][-260:]), 2)
        RS_Rating = round(rs_df[rs_df['Ticker']==stock].RS_Rating.tolist()[0])
        
        try:
            moving_average_200_20 = df["SMA_200"][-20]
        except Exception:
            moving_average_200_20 = 0

        # Condition 1: Current Price > 150 SMA and > 200 SMA
        condition_1 = currentClose > moving_average_150 > moving_average_200
        
        # Condition 2: 150 SMA and > 200 SMA
        condition_2 = moving_average_150 > moving_average_200

        # Condition 3: 200 SMA trending up for at least 1 month
        condition_3 = moving_average_200 > moving_average_200_20
        
        # Condition 4: 50 SMA> 150 SMA and 50 SMA> 200 SMA
        condition_4 = moving_average_50 > moving_average_150 > moving_average_200
           
        # Condition 5: Current Price > 50 SMA
        condition_5 = currentClose > moving_average_50
           
        # Condition 6: Current Price is at least 30% above 52 week low
        condition_6 = currentClose >= (1.3*low_of_52week)
           
        # Condition 7: Current Price is within 25% of 52 week high
        condition_7 = currentClose >= (.75*high_of_52week)
        
        # If all conditions above are true, add stock to exportList
        if(condition_1 and condition_2 and condition_3 and condition_4 and condition_5 and condition_6 and condition_7):
            exportList = exportList.append({'Stock': stock, "RS_Rating": RS_Rating ,"50 Day MA": moving_average_50, "150 Day Ma": moving_average_150, "200 Day MA": moving_average_200, "52 Week Low": low_of_52week, "52 week High": high_of_52week}, ignore_index=True)
    except Exception as e:
        print (e)
        print(f"Could not gather data on {stock}")

Lastly, we download the stocks to an excel file for convenience. To do this we sort our export list and then export the list to excel using Excel Writer
Printing the exportList shows us what the coloumns in the Excel File look like, this can be seen in the ouput below the code. 

In [7]:
exportList = exportList.sort_values(by='RS_Rating', ascending=False)
print('\n', exportList)
writer = ExcelWriter("ScreenOutput.xlsx")
exportList.to_excel(writer, "Sheet1")
writer.save()


      Stock RS_Rating  50 Day MA  150 Day Ma  200 Day MA  52 Week Low  \
23     CZR       100      91.25       74.75       66.76        16.91   
64      LB       100      59.53       45.31       40.46        10.03   
35     FCX        99      35.06       27.02       23.90         7.81   
46     GPS        99      29.19       23.55       21.39         6.54   
125   TSLA        99     692.07      629.33      561.60       136.61   
..     ...       ...        ...         ...         ...          ...   
14     BWA        71      47.13       41.80       41.20        25.17   
45   GOOGL        70    2127.68     1858.88     1778.86      1296.01   
38     FRC        70     170.42      147.04      138.37        92.13   
83    MXIM        70      92.65       85.50       81.21        50.77   
98     PNC        70     174.71      146.37      136.28        93.27   

     52 week High  
23         106.20  
64          68.29  
35          39.84  
46          34.82  
125        900.40  
..           